In [1]:
# Install pymgrid
!pip install git+https://github.com/Total-RD/pymgrid/
!pip install sklearn

  Cloning https://github.com/Total-RD/pymgrid/ to /tmp/pip-req-build-id267u0v
  Running command git clone -q https://github.com/Total-RD/pymgrid/ /tmp/pip-req-build-id267u0v


# Nouvelle section

In [2]:
# Libraries
import time # Necessary to evaluate frugality
import json # Necessary to export your results
import pickle # Necessary to load the data
import DiscreteEnvironment as DiscreteEnvironment # Imposed Discrete Environment
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np
import gym
import random
import matplotlib.pyplot as plt

In [3]:
# Open data

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)
    building_1.train_test_split()

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    building_2.train_test_split()

with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)
    building_3.train_test_split()

In [4]:
# Group building
buildings = [building_1, building_2, building_3]

In [5]:
# Global hyperparameters
def make_hyperparameters(C = 5,
                         alpha = None,
                         omega = 0.7,
                         discount_factor = 0.99,
                         epsilon = 0.99,
                         epsilon_min = 0.1,
                         epsilon_decay = 0.02,
                         epsilon_expo = False,
                         train_days = 4,
                         train_episodes = 200,
                         train_episodes_decay = 30,
                         max_steps = 24):
  return {
          "C" : C,
          "alpha" : alpha,
          "omega" : omega,
          "discount_factor" : discount_factor,
          "epsilon" : epsilon,
          "epsilon_min" : epsilon_min,
          "epsilon_decay" : epsilon_decay,
          "epsilon_expo" : epsilon_expo,
          "train_days" : train_days,
          "train_episodes" : train_episodes,
          "train_episodes_decay" : train_episodes_decay,
          "max_steps" : max_steps
  }

In [6]:
# Test
def test(env, QA, QB, testing=False):
    state = env.reset(testing=testing)
    done = False
    total_cost = 0
    while not done:
        actionA = np.argmax(QA[state])
        actionB = np.argmax(QB[state])
        if QA[state][actionA] > QB[state][actionB]:
            action = actionA
        else:
            action = actionB
        obs, reward, done, info = env.step(action)
        total_cost += reward
    return total_cost

In [7]:
# Test ML model
def test_ml(env, clf, testing=False):
    state = env.reset(testing=testing)
    done = False
    total_cost = 0
    while not done:
        action = clf.predict(np.array(state).reshape(1, 2))
        obs, reward, done, info = env.step(action)
        total_cost += reward
    return total_cost

In [8]:
# Training the agent

def train(p, env, debug=False):

    # Display the state space
    if debug:
        print("Action Space {}".format(env.action_space))
        print("State Space {}".format(env.observation_space))

    # Intialize the Q table
    QA, QB = {}, {}

    # Reset discoveries
    QA_count, QB_count = {}, {}

    # Use the first reward to initialize
    C = p["C"]

    # Initialize epsilon and alpha
    epsilon = p["epsilon"]

    # Custom alpha decay (https://www.jmlr.org/papers/volume5/evendar03a/evendar03a.pdf)
    """def get_alpha(s,a):
        if (s in Q_count) and (Q_count[s][a] > 0):
            return 1.0 / np.power(Q_count[s][a], p["omega"])
        else:
            return 0.0"""
    
    def get_alpha(s,a, Q_count):
        if (s in Q_count) and (Q_count[s][a] > 0):
            return 1.0 / np.power(Q_count[s][a], p["omega"])
        else:
            return 0.0

    # Train on different days
    training_rewards = []

    train_episodes = p["train_episodes"]
    for day in range(p["train_days"]):
        # print("Day {}/{}".format(day+1,p["train_days"]))

        # Creating lists to keep track of reward and epsilon values
        day_training_rewards = []

        for episode in range(train_episodes):
            # print("Episode {}/{}".format(episode+1,train_episodes))

            #Reseting the environment each time as per requirement
            state = tuple(env.reset())
            if not (state in QA):
                QA[state] = [C for i in range(env.Na)]
                QB[state] = [C for i in range(env.Na)]
            if not (state in QA_count):
                QA_count[state] = [0 for i in range(env.Na)]
                QB_count[state] = [0 for i in range(env.Na)]

            # Starting the tracker for the rewards
            total_training_rewards = 0

            # Choosing an action given the states based on a random number
            exp_exp_tradeoff = np.random.random()

            ### STEP 2: SECOND option for choosing the initial action - exploit
            # If the random number is larger than epsilon: employing exploitation
            # and selecting best action
            if exp_exp_tradeoff < (1 - epsilon):
                actionA = np.argmax(QA[state])
                actionB = np.argmax(QB[state])
                if QA[state][actionA] > QB[state][actionB]:
                    action = actionA
                else:
                    action = actionB
            ### STEP 2: FIRST option for choosing the initial action - explore
            # Otherwise, employing exploration: choosing a random action
            else:
                action = np.random.choice(env.Na) # env.action_space.sample()

            for step in range(p["max_steps"]):

                ### STEPs 3 & 4: performing the action and getting the reward
                # Taking the action and getting the reward and outcome state
                new_state, reward, done, info = env.step(action)
                new_state = tuple(new_state)

                ### STEP 5: update the Q-table

                # Check if values are in the table
                if not (state in QA):
                    QA[state] = [C for i in range(env.Na)]
                    QB[state] = [C for i in range(env.Na)]
                if not (state in QA_count):
                    QA_count[state] = [0 for i in range(env.Na)]
                    QB_count[state] = [0 for i in range(env.Na)]
                if not (new_state in QA):
                    QA[new_state] = [C for i in range(env.Na)]
                    QB[new_state] = [C for i in range(env.Na)]

                # Set dynamic alpha
                if p["alpha"]:
                    alpha = p["alpha"]

                # 50% chance to update QA (or QB)
                choice = np.random.random()

                # Updating Q's
                if choice >= 0.5:
                    new_action = np.argmax(QA[new_state])

                    if not p["alpha"]:
                        alpha = get_alpha(new_state, action, QA_count)

                    # Updating the Q-table using the Bellman equation
                    if step == p["max_steps"] - 1:
                        QA[state][action] += alpha * (reward - QA[state][action])
                    else:
                        QA[state][action] = (1 - alpha) * QA[state][action] + alpha * (reward + p["discount_factor"] * QA[new_state][new_action] - QA[state][action])
                    QA_count[state][action] += 1
                  
                elif choice < 0.5:
                    new_action = np.argmax(QB[new_state])

                    if not p["alpha"]:
                        alpha = get_alpha(new_state, action, QB_count)

                    # Updating the Q-table using the Bellman equation
                    if step == p["max_steps"] - 1:
                        QB[state][action] += alpha * (reward - QB[state][action])
                    else:
                        QB[state][action] = (1 - alpha) * QB[state][action] + alpha * (reward + p["discount_factor"] * QB[new_state][new_action] - QB[state][action])
                    QB_count[state][action] += 1

                # Increasing our total reward and updating the state
                total_training_rewards -= reward
                state = new_state
                action = new_action

                # Ending the episode
                if done == True:
                    if debug:
                        print ("Total reward for episode {}: {}".format(episode, total_training_rewards))
                    break

            # Cutting down on exploration by reducing the epsilon
            if p["epsilon_expo"]:
              epsilon = p["epsilon_min"] + (p["epsilon"] - p["epsilon_min"]) * np.exp(-p["epsilon_decay"] * episode)
            else:
              epsilon = max(p["epsilon_min"], epsilon - epsilon * p["epsilon_decay"])

            # Adding the total reward and reduced epsilon values
            day_training_rewards.append(total_training_rewards)

        # Decrease the number of steps
        train_episodes -= p["train_episodes_decay"]
        training_rewards += day_training_rewards

        if debug:
            print ("Training score over time (Day {}): ".format(day) + str(sum(training_rewards)/train_episodes))

    return training_rewards, QA, QB, QA_count, QB_count

In [9]:
# Build a decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import random

def build_clf(p, QA, QB, QA_count, QB_count):

  # Build dataset
  X = []
  y = []
  for s in QA.keys():
    actionA = np.argmax(QA[s])
    actionB = np.argmax(QB[s])

    # Choose the best action
    if QA[s][actionA] > QB[s][actionB]:
        action, orig = actionA, 1
    elif QA[s][actionA] < QB[s][actionB]:
        action, orig = actionB, 0
    else:
        action, orig = random.choice([(actionA,1), (actionB,0)])

    # If unitialized, choose the another
    if (orig == 1) and not ((s in QA_count and QA_count[s][action] > 0) or (QA[s][action] != p['C'])):
        action, orig = actionB, 0
    elif (orig == 0) and not ((s in QB_count and QB_count[s][action] > 0) or (QB[s][action] != p['C'])):
        action, orig = actionA, 1    

    # Append to dataset
    if (orig == 1) and ((s in QA_count and QA_count[s][action] > 0) or (QA[s][action] != p['C'])):
      y.append(action)
      X.append(np.array(s))
    if (orig == 0) and ((s in QB_count and QB_count[s][action] > 0) or (QB[s][action] != p['C'])):
      y.append(action)
      X.append(np.array(s))

  # Build and train model
  clf = DecisionTreeClassifier(max_depth=4, random_state=0)
  clf.fit(X, y)

  return clf

In [10]:
# Full benchmark
import time

def benchmark(p):
  building_environments = [DiscreteEnvironment.Environment(env_config={'building':buildings[i]}) for i in range(3)]

  # Train on the first building
  clfs = [None, None, None]
  train_start = time.process_time()
  for i,building_env in enumerate(building_environments):
    training_rewards, QA, QB, QA_count, QB_count = train(p, building_env)
    clfs[i] = build_clf(p, QA, QB, QA_count, QB_count)
  train_end = time.process_time()

  # Test our model on train data
  scores_train = [0,0,0]
  for i,building_env in enumerate(building_environments):
    scores_train[i] = -1 * test_ml(building_env, clfs[i], testing=False)

  # Test our model on test data
  scores_test = [0,0,0]
  test_start = time.process_time()
  for i,building_env in enumerate(building_environments):
    scores_test[i] = -1 * test_ml(building_env, clfs[i], testing=True)
  test_end = time.process_time()

  # Usefull quantities
  train_frugality = train_end - train_start
  test_frugality = test_end - test_start
  frugality = train_frugality + test_frugality
  total_cost = scores_test

  # Make results
  final_results = {
      "building_1_performance" : total_cost[0],
      "building_2_performance" : total_cost[1],
      "building_3_performance" : total_cost[2],
      "frugality" : frugality,
  }

  return final_results

In [11]:
# Benchmark our model

best_param = {
     'C': 5,
     'alpha': 0.22631578947368422,
     'discount_factor': 1.0,
     'epsilon': 0.99,
     'epsilon_decay': 4.894736842105263,
     'epsilon_expo': True,
     'epsilon_min': 0.1,
     'max_steps': 24,
     'omega': 0.7289473684210527,
     'train_days': 1,
     'train_episodes': 30,
     'train_episodes_decay': 30
}
random.seed(42)
np.random.seed(42)
final_results = benchmark(best_param)

In [12]:
# Save to JSON
with open('corai.txt', 'w') as json_file:
    json.dump(final_results, json_file)

In [13]:
# Print results
import pprint

pprint.pprint(final_results)

{'building_1_performance': 4089.2658357035007,
 'building_2_performance': 13630.886119011686,
 'building_3_performance': 16741.597568815654,
 'frugality': 5.1903507069999995}
